In [2]:
!ls

cfgs	 _init_paths.py   models	    script.txt
data	 _init_paths.pyc  __pycache__	    Take a look at ImageNet.ipynb
demo.py  lib		  README.md	    test_net.py
images	 LICENSE	  requirements.txt  trainval_net.py


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import xml.dom.minidom as minidom
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader

In [2]:
from datasets.imagenet import imagenet

In [3]:
dataset = 'imagenet'
imdb_name = 'imagenet_train'
imdbval_name = 'imagenet_val'

In [4]:
split = 'train'#train val val1 val2 test
name = 'imagenet_{}'.format(split)
devkit_path = 'data/imagenet/ILSVRC/devkit'
data_path = 'data/imagenet/ILSVRC'

In [5]:
db_handle = (lambda split=split, devkit_path=devkit_path, data_path=data_path: imagenet(split,devkit_path,data_path))

In [6]:
print(type(db_handle))

<type 'function'>


In [7]:
import scipy.io as sio

In [8]:
synsets_image = sio.loadmat(os.path.join(devkit_path,'data','meta_det.mat'))

In [9]:
synsets_video = sio.loadmat(os.path.join(devkit_path,'data','meta_vid.mat'))

In [10]:
print(synsets_image.keys())

['__version__', '__header__', 'synsets', '__globals__']


In [11]:
print(synsets_image)

{'__version__': '1.0', '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun May 24 19:54:11 2015', 'synsets': array([[(array([[1]], dtype=uint8), array([u'n02672831'], dtype='<U9'), array([u'accordion'], dtype='<U9'), array([u'a portable box-shaped free-reed instrument; the reeds are made to vibrate by air from the bellows controlled by the player)'],
      dtype='<U123')),
        (array([[2]], dtype=uint8), array([u'n02691156'], dtype='<U9'), array([u'airplane'], dtype='<U8'), array([u' an aircraft powered by propellers or jets'], dtype='<U42')),
        (array([[3]], dtype=uint8), array([u'n02219486'], dtype='<U9'), array([u'ant'], dtype='<U3'), array([u' emmet, pismire'], dtype='<U15')),
        (array([[4]], dtype=uint8), array([u'n02419796'], dtype='<U9'), array([u'antelope'], dtype='<U8'), array([u' a graceful animal with long legs and horns directed upward and backward'],
      dtype='<U72')),
        (array([[5]], dtype=uint8), array([u'n07739125'], dtype='<U

In [12]:
syn_I = synsets_image['synsets']
syn_V = synsets_video['synsets']

In [13]:
print(type(syn_I))

<type 'numpy.ndarray'>


In [14]:
print(syn_I.shape)

(1, 200)


#### synsets are in order of: ID(int), WNID(string), Name(string), Description(string).

In [15]:
print(syn_I[0,0])

(array([[1]], dtype=uint8), array([u'n02672831'], dtype='<U9'), array([u'accordion'], dtype='<U9'), array([u'a portable box-shaped free-reed instrument; the reeds are made to vibrate by air from the bellows controlled by the player)'],
      dtype='<U123'))


In [16]:
print(syn_V[0,0])

(array([[1]], dtype=uint8), array([u'n02691156'], dtype='<U9'), array([u'airplane'], dtype='<U8'), array([u' an aircraft powered by propellers or jets'], dtype='<U42'))


#### find the wnids in both VID and DET

In [17]:
vid_wnids = [syn_V[0,i][1] for i in range(30)]
print(vid_wnids)

[array([u'n02691156'], dtype='<U9'), array([u'n02419796'], dtype='<U9'), array([u'n02131653'], dtype='<U9'), array([u'n02834778'], dtype='<U9'), array([u'n01503061'], dtype='<U9'), array([u'n02924116'], dtype='<U9'), array([u'n02958343'], dtype='<U9'), array([u'n02402425'], dtype='<U9'), array([u'n02084071'], dtype='<U9'), array([u'n02121808'], dtype='<U9'), array([u'n02503517'], dtype='<U9'), array([u'n02118333'], dtype='<U9'), array([u'n02510455'], dtype='<U9'), array([u'n02342885'], dtype='<U9'), array([u'n02374451'], dtype='<U9'), array([u'n02129165'], dtype='<U9'), array([u'n01674464'], dtype='<U9'), array([u'n02484322'], dtype='<U9'), array([u'n03790512'], dtype='<U9'), array([u'n02324045'], dtype='<U9'), array([u'n02509815'], dtype='<U9'), array([u'n02411705'], dtype='<U9'), array([u'n01726692'], dtype='<U9'), array([u'n02355227'], dtype='<U9'), array([u'n02129604'], dtype='<U9'), array([u'n04468005'], dtype='<U9'), array([u'n01662784'], dtype='<U9'), array([u'n04530566'], dtype

In [18]:
used_wnid_name = []
for i in range(0,200):
    wnid = syn_I[0,i][1]
    name = syn_I[0,i][2]
    if wnid in vid_wnids:
        #print(wnid,name)
        used_wnid_name.append((wnid,name))
print(len(used_wnid_name))

30


#### text file for different classes.

In [19]:
vtmp_index = [] # label as 1.
vtmp_index0 = [] # label as 0.
vtmp_index_1 = [] # label as -1.
for i in range(1,201):
    image_set_file = os.path.join(data_path,'ImageSets','DET','train_'+str(i)+'.txt')
    print(image_set_file,os.path.exists(image_set_file))
    with open(image_set_file,'r') as f:
        tmp_index = [x for x in f.readlines()]
        vtmp = []
        for line in tmp_index:
            line = line.strip()
            line = line.split(' ')
            if line[1] == '1':
                vtmp_index.append(data_path+'/Data/DET/train/'+line[0])
            elif line[1] == '0':
                vtmp_index0.append(data_path+'/Data/DET/train/'+line[0])
            elif line[1] == '-1':
                vtmp_index_1.append(data_path+'/Data/DET/train/'+line[0])
            else:
                raise Exception('Error.')

data/imagenet/ILSVRC/ImageSets/DET/train_1.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_2.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_3.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_4.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_5.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_6.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_7.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_8.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_9.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_10.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_11.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_12.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_13.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_14.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_15.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_16.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_17.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_18.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_19.txt True
da

data/imagenet/ILSVRC/ImageSets/DET/train_155.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_156.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_157.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_158.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_159.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_160.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_161.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_162.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_163.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_164.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_165.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_166.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_167.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_168.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_169.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_170.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_171.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_172.txt True
data/imagenet/ILSVRC/ImageSe

In [20]:
print(len(vtmp_index),len(vtmp_index0),len(vtmp_index_1),'sum:',len(vtmp_index)+len(vtmp_index0)+len(vtmp_index_1))

373852 1946 13010980 sum: 13386778


#### There are still some overlaps between input files.

In [21]:
len(np.unique(vtmp_index))

332764

##### what about trian.txt val.txt and test.txt. It is < image_index + id_number >

In [22]:
print('Training set:')
image_set_file = os.path.join(data_path,'ImageSets','DET','train.txt')
print(image_set_file,os.path.exists(image_set_file))
with open(image_set_file,'r') as f:
    tmp_index = [x for x in f.readlines()]
print(len(tmp_index))
print(tmp_index[0])

Training set:
data/imagenet/ILSVRC/ImageSets/DET/train.txt True
456567
ILSVRC2013_train_extra0/ILSVRC2013_train_00000001 1



In [23]:
print('Val set:')
image_set_file = os.path.join(data_path,'ImageSets','DET','val.txt')
print(image_set_file,os.path.exists(image_set_file))
with open(image_set_file,'r') as f:
    tmp_index = [x for x in f.readlines()]
print(len(tmp_index))

Val set:
data/imagenet/ILSVRC/ImageSets/DET/val.txt True
20120


### Examine Video set

In [24]:
vtmp_index = [] # label as 1.
vtmp_index0 = [] # label as 0.
vtmp_index_1 = [] # label as -1.
for i in range(1,31):
    image_set_file = os.path.join(data_path,'ImageSets','VID','train_'+str(i)+'.txt')
    print(image_set_file,os.path.exists(image_set_file))
    with open(image_set_file,'r') as f:
        tmp_index = [x for x in f.readlines()]
        vtmp = []
        for line in tmp_index:
            line = line.strip()
            line = line.split(' ')
            assert os.path.exists(data_path+'/Data/VID/train/'+line[0])
            if line[1] == '1':
                vtmp_index.append(data_path+'/Data/VID/train/'+line[0])
            elif line[1] == '0':
                vtmp_index0.append(data_path+'/Data/VID/train/'+line[0])
            elif line[1] == '-1':
                vtmp_index_1.append(data_path+'/Data/VID/train/'+line[0])
            else:
                raise Exception('Error.')

data/imagenet/ILSVRC/ImageSets/VID/train_1.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_2.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_3.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_4.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_5.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_6.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_7.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_8.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_9.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_10.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_11.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_12.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_13.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_14.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_15.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_16.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_17.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_18.txt True
data/imagenet/ILSVRC/ImageSets/VID/train_19.txt True
da

#### All the images in video set are fully annotated.

In [25]:
print(len(vtmp_index),len(vtmp_index0),len(vtmp_index_1),'sum:',len(vtmp_index)+len(vtmp_index0)+len(vtmp_index_1))

4381 0 0 sum: 4381


In [26]:
print(vtmp_index[0])

data/imagenet/ILSVRC/Data/VID/train/ILSVRC2015_VID_train_0000/ILSVRC2015_train_00030000


In [28]:
#def get_vid_train_data_statistics(homePath = 'data/imagenet/ILSVRC/'):
homePath = 'data/imagenet/ILSVRC/'
dataPath = os.path.join(homePath,'Data','VID','train')
setPath = os.path.join(homePath,'ImageSets','VID')
filenames_all = []
vid_num = 0
vid_counter = np.zeros(30)
for vid_idx in range(1,31):
    image_set_file = os.path.join(homePath,'ImageSets','VID','train_'+str(vid_idx)+'.txt')
    filenames = []
    with open(image_set_file) as f:
        items = [x.split(' ')[0] for x in f.readlines()]
        for idx in range(len(items)):
            vid_num += 1
            vid_counter[vid_idx-1] += 1
            item = items[idx]
            item_dir = os.path.join(dataPath, item)
            fns = [fn for fn in os.listdir(item_dir) if 'JPEG' in fn]
            assert fns>0
            filenames.extend(fns)
    assert len(filenames)>0
    print(syn_V[0,vid_idx-1][2],len(filenames))
    filenames_all.extend(filenames)
print('total vid number:',vid_num)
print('total images:',len(filenames_all))
print('number of videos for each category:',vid_counter)
get_vid_train_data_statistics()

[u'airplane'] 57418
[u'antelope'] 28333
[u'bear'] 39745
[u'bicycle'] 32881
[u'bird'] 90177
[u'bus'] 24366
[u'car'] 107462
[u'cattle'] 24310
[u'dog'] 107450
[u'domestic_cat'] 58921
[u'elephant'] 34761
[u'fox'] 32556
[u'giant_panda'] 37878
[u'hamster'] 30705
[u'horse'] 35537
[u'lion'] 20648
[u'lizard'] 35604
[u'monkey'] 47367
[u'motorcycle'] 27189
[u'rabbit'] 32325
[u'red_panda'] 34572
[u'sheep'] 17807
[u'snake'] 33613
[u'squirrel'] 50053
[u'tiger'] 19381
[u'train'] 104469
[u'turtle'] 45541
[u'watercraft'] 38864
[u'whale'] 43769
[u'zebra'] 28984
total vid number: 4381
total images: 1322686
number of videos for each category: [172. 112. 114. 267. 292.  70. 452.  73. 458. 183.  79. 102. 116.  80.
 139.  64.  92. 129. 149.  75. 134.  76.  96. 132.  56. 158. 118. 154.
 139. 100.]
[u'airplane'] 57418
[u'antelope'] 28333
[u'bear'] 39745
[u'bicycle'] 32881
[u'bird'] 90177
[u'bus'] 24366
[u'car'] 107462
[u'cattle'] 24310
[u'dog'] 107450
[u'domestic_cat'] 58921
[u'elephant'] 34761
[u'fox'] 32556


In [50]:
def cal_vid_number(vid_counter, frames_per_vid = 15, max_vid_per_category = 100):
    return np.minimum(vid_counter, max_vid_per_category)*frames_per_vid
print(cal_vid_number(vid_counter))
print(np.sum(cal_vid_number(vid_counter)))

[1500. 1500. 1500. 1500. 1500. 1050. 1500. 1095. 1500. 1500. 1185. 1500.
 1500. 1200. 1500.  960. 1380. 1500. 1500. 1125. 1500. 1140. 1440. 1500.
  840. 1500. 1500. 1500. 1500. 1500.]
41415.0


In [72]:
####VID object info####
_wnid = (0,)
_classes = ('__background__',)
for i in xrange(30):
    _classes = _classes + (synsets_video['synsets'][0][i][2][0],)
    _wnid = _wnid + (synsets_video['synsets'][0][i][1][0],)

_wnid_to_ind = dict(zip(_wnid, xrange(31)))
_class_to_ind = dict(zip(_classes, xrange(31)))
#######################
def read_annotation(dataPath):
    filename = dataPath.replace('Data','Annotations').replace('JPEG','xml')
    assert os.path.exists(filename),'%s'%(filename)
    # print 'Loading: {}'.format(filename)
    def get_data_from_tag(node, tag):
        return node.getElementsByTagName(tag)[0].childNodes[0].data

    with open(filename) as f:
        data = minidom.parseString(f.read())

    objs = data.getElementsByTagName('object')
    num_objs = len(objs)
    
    boxes = np.zeros((num_objs, 4), dtype=np.int32)
    gt_classes = np.zeros(num_objs,dtype=np.int32)
    # Load object bounding boxes into a data frame.
    for ix, obj in enumerate(objs):
        x1 = float(get_data_from_tag(obj, 'xmin'))
        y1 = float(get_data_from_tag(obj, 'ymin'))
        x2 = float(get_data_from_tag(obj, 'xmax'))
        y2 = float(get_data_from_tag(obj, 'ymax'))
        cls = _wnid_to_ind[
                str(get_data_from_tag(obj, "name")).lower().strip()]
        boxes[ix, :] = [x1, y1, x2, y2]
        gt_classes[ix] = cls
    return boxes, gt_classes

In [112]:
def watch_vid_train_data(vid_idx, idx, homePath = 'data/imagenet/ILSVRC/', show_class=True, save_vid=False, dest_vid_path = 'video.avi'):
    dataPath = os.path.join(homePath,'Data','VID','train')
    setPath = os.path.join(homePath,'ImageSets','VID')
    assert vid_idx>0 and vid_idx<31
    image_set_file = os.path.join(homePath,'ImageSets','VID','train_'+str(vid_idx)+'.txt')
    with open(image_set_file) as f:
        items = [x.split(' ')[0] for x in f.readlines()]
        assert idx>=0 and idx<len(items), print(len(items))
        item = items[idx]
        item_dir = os.path.join(dataPath, item)
        filenames = [fn for fn in os.listdir(item_dir) if 'JPEG' in fn]
        assert len(filenames)>0
        print(len(filenames))
        height , width , _ =  cv2.imread(os.path.join(item_dir,filenames[0])).shape
        
        while height>800 or width > 800:
            height = int(height/2)
            width = int(width/2)
        
        if save_vid:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            video = cv2.VideoWriter(dest_vid_path, fourcc,20.0,(width,height))
        for i in range(len(filenames)):
            sys.stdout.write('%d/%d \r'%(i+1,len(filenames)))
            sys.stdout.flush()
            img = cv2.imread(os.path.join(item_dir,'%06d.JPEG'%(i)))
            boxes, gt_classes = read_annotation(os.path.join(item_dir,'%06d.JPEG'%(i)))
            
            for obj_id in range(len(gt_classes)):
                bbox = boxes[obj_id,:]
                class_name = _classes[gt_classes[obj_id]]
                
                cv2.rectangle(img, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0, 204, 0), 2)
                
                cv2.putText(img, '%s' % (class_name), (bbox[0], bbox[1] + 15), cv2.FONT_HERSHEY_PLAIN,
                            1.0, (0, 0, 255), thickness=1)
            
            if img.shape[0]!=height:
                img = cv2.resize(img,(width,height))
            if save_vid:
                video.write(img)
            else:
                cv2.imshow('image',img)
                cv2.waitKey(50)
        video.release()
        cv2.destroyAllWindows()

In [113]:
watch_vid_train_data(1,35,save_vid=True,dest_vid_path='video.avi')

199


In [114]:
def getEgTrainDataVids():
    save_dir = 'eg_train_vid'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    for i in range(1,31):
        try:
            for j in range(0,1000,10):
                watch_vid_train_data(i,j,save_vid=True,dest_vid_path=os.path.join(save_dir,'%d-%d.avi'%(i,j)))
        except:
            print('j out of range.')

In [115]:
getEgTrainDataVids()

1345
1735/1345 
963/173 
11116 
7511/1111 
35/75 
93/35 
15993 
229/159 
15322 
783/153 
32378 
580/323 
159/580 
914/159 
534/914 
18453 
197/184 
172/197 
j out of range.
647
807/647 
18380 
216/183 
100/216 
267/100 
907/267 
39090 
573/390 
973/573 
10397 
435/103 
112/435 
j out of range.
550
430/550 
18443 
118/184 
638/118 
755/638 
536/755 
726/536 
64672 
178/646 
228/178 
468/228 
114/468 
j out of range.
72
92/72 
99/92 
61/99 
22361 
793/223 
66/79 
43/66 
16043 
113/160 
653/113 
19935 
2893/1993 
47/28 
40/47 
18/40 
56/18 
11756 
607/117 
12560 
615/125 
24/61 
41624 
566/416 
88/56 
82/88 
25082 
267/250 
j out of range.
260
750/260 
87/75 
11687 
174/116 
854/174 
10885 
325/108 
150/325 
650/150 
18/65 
24/18 
56/24 
14856 
153/148 
290/153 
145/290 
119/145 
610/119 
460/610 
725/460 
464/725 
164/464 
108/164 
522/108 
682/522 
22568 
900/225 
435/900 
845/435 
29284 
j out of range.
380
575/380 
448/575 
768/448 
10376 
773/103 
10447 
7044/1044 
j out of range.
15

#### check val set.

In [67]:
vtmp_index = [] # label as 1.
vtmp_index0 = [] # label as 0.
vtmp_index_1 = [] # label as -1.
for i in range(1,31):
    image_set_file = os.path.join(data_path,'ImageSets','VID','train_'+str(i)+'.txt')
    print(image_set_file,os.path.exists(image_set_file))
    with open(image_set_file,'r') as f:
        tmp_index = [x for x in f.readlines()]
        vtmp = []
        for line in tmp_index:
            line = line.strip()
            line = line.split(' ')
            if line[1] == '1':
                vtmp_index.append(data_path+'/Data/DET/train/'+line[0])
            elif line[1] == '0':
                vtmp_index0.append(data_path+'/Data/DET/train/'+line[0])
            elif line[1] == '-1':
                vtmp_index_1.append(data_path+'/Data/DET/train/'+line[0])
            else:
                raise Exception('Error.')

## Examine the annotation files.